In [ ]:
%load_ext autoreload
%autoreload 2

from pprint import pprint

from read_excel import *
from compute_state import *
from collections import namedtuple
from custom_types import *
from prices.balanz_fci_price import *
from timeseries_analysis import *
from datetime import datetime
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [ ]:
# Example usage:
file_path = 'private/movimientos.xlsx'  # Set the filename to 'ordenes.xlsx'
df = read_excel_to_dataframe(file_path)
print(df.loc[0])

In [ ]:
# Create a State tuple containing the Balance tuple
current_state = State(Fondos=tuple(), Cedears=tuple(), Bonos=tuple(), Corporativos=tuple())

states_over_time = {}

# Iterate through all rows of the DataFrame
for index, row in df[::-1].iterrows():
    
    # Call the update_state function for each row
    next_state = update_state(row, current_state)
    
    # Update the current state to the next state
    current_state = next_state

    states_over_time[row['Concertacion'].date()] = current_state

current_state

In [ ]:
fondos_over_time = extract_instrument_from_state(states_over_time, 'Fondos')

In [ ]:
dict_to_dataframe(fondos_over_time)

In [ ]:
FCIs_amounts = fill_missing_days(dict_to_dataframe(fondos_over_time)).drop(['BCMMA', 'BCAHA'], axis=1)
FCIs_amounts

In [ ]:
FCIs_prices = get_fcis_price_df(FCIs_amounts.columns, FCIs_amounts.index)
FCIs_prices

In [ ]:
FCIs_values = FCIs_amounts*FCIs_prices
FCIs_values

In [ ]:
pd.options.plotting.backend = "plotly"

# Plot using Plotly Express
fig = FCIs_values.plot()
fig.layout.height = 500
fig.layout.width = 1000
fig.show()

In [ ]:
dolar_mep = get_dolar_price_df(FCIs_amounts.index)
dolar_mep

In [ ]:
FCIs_values_dolar = FCIs_values.div(dolar_mep.iloc[:, 0], axis=0)
FCIs_values_dolar

In [ ]:
# Plot using Plotly Express
fig = FCIs_values_dolar.plot()
fig.layout.height = 500
fig.layout.width = 1000
fig.show()